# core

> Fill in a module description here

In [ ]:
#| default_exp

In [3]:
#| hide
from nbdev.showdoc import *

import os

from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

import google

In [11]:
#| export
class GetCreds_AuthException(Exception):
    def __init__(self):
        super().__init__( "must pass `token_file_path` or `creds` of type google.oauth2.credentials.Credentials class to get_creds()")

class GetCreds_InvalidTokenPath(Exception):
    def __init__(self):
        super().__init__( "invalid `token_file_path`.  Correct or pass `creds` of type google.oauth2.credentials.Credentials class to get_creds()")


In [12]:
# | export

def get_creds(token_file_path: str = None,
              credentials_file_path: str = None,
              creds: google.oauth2.credentials.Credentials = None):
    """generate project and associated creds for google drive project here, https://console.cloud.google.com/apis/credentials/oauthclient"""

    if not creds and not token_file_path:
        raise GetCreds_AuthException()

    if not creds and not os.path.exists(token_file_path):
        raise GetCreds_InvalidTokenPath()

    # If modifying these scopes, delete the file token.json.
    SCOPES = ['https://www.googleapis.com/auth/drive',
              'https://www.googleapis.com/auth/drive.file',
              'https://www.googleapis.com/auth/drive.metadata.readonly',
              'https://www.googleapis.com/auth/drive.readonly']

    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.

    if not creds:
        creds = Credentials.from_authorized_user_file(token_file_path, SCOPES)

    # If there are no (valid) credentials available, let the user log in.
    if not creds.valid:
        if creds.expired and creds.refresh_token:
            creds.refresh(Request())
        
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                credentials_file_path, SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for the next run
        with open(token_file_path, 'w') as token:
            token.write(creds.to_json())
    
    return creds

#### sample implementation of get_creds

In [ ]:
from google.oauth2.credentials import Credentials

Credentials.from_authorized_user_file(token_file_path, SCOPES)


def get_creds(token_file_path: str = None,
              credentials_file_path: str = None,
              creds: google.oauth2.credentials.Credentials = None)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()